In [1]:
#import Libraries
import pandas as pd
import json

In [10]:
with open('t20_json_files/t20_wc_match_results.json') as f:
    data = json.load(f)

df_match = pd.DataFrame(data[0]['matchSummary'])
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


#### Use scorecard as a match id to link with other table

In [28]:
df_match.rename({'scorecard': 'match_id'}, axis = 1, inplace = True)
df_match.head()

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


#### Create a match ids dictionary that maps team names to a unique match id. This will be useful later on to link with other tables

In [53]:
match_ids_dict = {}

for index, row in df_match.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    match_ids_dict[key1] = row['match_id']
    match_ids_dict[key2] = row['match_id']

## Batting Summary

In [19]:
with open('t20_json_files/t20_wc_batting_summary.json') as f:
    data = json.load(f)
    
    #since we need to add everything as a list
    all_records = []
    
    for rec in data:
        all_records.extend(rec['battingSummary'])
        
df_batting = pd.DataFrame(all_records)
df_batting.tail()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
694,Pakistan Vs England,England,3,Phil Salt,c Iftikhar Ahmed b Haris Rauf,10,9,2,0,111.11
695,Pakistan Vs England,England,4,Ben Stokes,,52,49,5,1,106.12
696,Pakistan Vs England,England,5,Harry Brook,c Shaheen Shah Afridi b Shadab Khan,20,23,1,0,86.95
697,Pakistan Vs England,England,6,Moeen Ali,b Mohammad Wasim,19,13,3,0,146.15
698,Pakistan Vs England,England,7,Liam Livingstone,,1,1,0,0,100.00


In [22]:
#We need to change the dismissal to out/not_out
#add another column and apply conditions

df_batting["Out/Not_Out"] = df_batting.dismissal.apply(lambda x: 'OUT' if len(x)>0 else "NOT OUT")
df_batting.head(15)

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR,Out/Not_Out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00,OUT
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00,OUT
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66,OUT
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33,OUT
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33,OUT
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,run out (Gunathilaka/â€ Mendis),44,28,4,0,157.14,OUT
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,c â€ Mendis b Theekshana,0,1,0,0,0.00,OUT
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,,31,16,2,2,193.75,NOT OUT
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,c Smit b Shikongo,9,10,1,0,90.00,OUT
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,c â€ Green b Wiese,6,6,0,0,100.00,OUT


In [25]:
#Drop dismissal column

df_batting.drop('dismissal',axis = 1, inplace = True)
df_batting.head(15)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/Not_Out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,OUT
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,OUT
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,NOT OUT
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,OUT
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,6,6,0,0,100.00,OUT


In [27]:
#remove specialcharacters

df_batting['batsmanName'] = df_batting['batsmanName'].apply(lamda x: x.replace('â€', ""))
df_batting['batsmanName'] = df_batting['batsmanName'].apply(lamda x: x.replace('\xa0', ""))

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/Not_Out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,OUT
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,OUT
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,NOT OUT
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,OUT
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,6,6,0,0,100.00,OUT


In [31]:
df_batting["match_id"] = df_batting['match'].map(match_ids_dict)

df_batting.head()

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/Not_Out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT,T20I # 1823


In [39]:
df_batting.to_csv('batting_summary.csv', index = False)

#### Bowling Summary

In [44]:
with open('t20_json_files/t20_wc_bowling_summary.json') as f:
    data = json.load(f)

all_records = []
    
    for rec in data:
        all_records.extend(rec['bowlingSummary'])
df_bowling = pd.DataFrame(all_records)

In [55]:
df_bowling.tail(10)

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
490,Pakistan Vs England,England,Sam Curran,4,0,12,3,3.00,15,0,0,0,0,T20I # 1879
491,Pakistan Vs England,England,Adil Rashid,4,1,22,2,5.50,10,1,0,1,0,T20I # 1879
492,Pakistan Vs England,England,Chris Jordan,4,0,27,2,6.75,9,3,0,0,0,T20I # 1879
493,Pakistan Vs England,England,Liam Livingstone,1,0,16,0,16.00,1,1,1,1,0,T20I # 1879
494,Pakistan Vs England,Pakistan,Shaheen Shah Afridi,2.1,0,13,1,6.00,6,1,0,0,0,T20I # 1879
495,Pakistan Vs England,Pakistan,Naseem Shah,4,0,30,0,7.50,15,3,1,1,0,T20I # 1879
496,Pakistan Vs England,Pakistan,Haris Rauf,4,0,23,2,5.75,13,3,0,1,0,T20I # 1879
497,Pakistan Vs England,Pakistan,Shadab Khan,4,0,20,1,5.00,10,1,0,0,0,T20I # 1879
498,Pakistan Vs England,Pakistan,Mohammad Wasim,4,0,38,1,9.50,5,5,0,2,0,T20I # 1879
499,Pakistan Vs England,Pakistan,Iftikhar Ahmed,0.5,0,13,0,15.60,0,1,1,0,0,T20I # 1879


In [51]:
#map "match" with "match_ids_dict"

df_bowling['match_id'] = df_bowling['match'].map(match_ids_dict)
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [56]:
df_bowling.to_csv("bowling.csv", index= False)

### Player Info 

In [58]:
with open('t20_wc_player_info.json') as f:
    data = json.load(f)
    
df_players = pd.DataFrame(data)
df_players

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
...,...,...,...,...,...,...
214,Tabraiz Shamsi,South Africa,Right hand Bat,Left arm Wrist spin,Bowler,A left-arm wristspin bowler with ample variati...
215,Phil Salt,England,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,Phil Salt's fearless ball-striking at the top ...
216,Chris Jordan,England,Right hand Bat,Right arm Fast medium,Bowler,"Chris Jordan, a fast bowler with a level-heade..."
217,Regis Chakabva(c)â€,Zimbabwe,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,"A doughty wicketkeeper-batsman, Regis Chakabva..."


### Remove Special Characters

In [59]:
df_players['name'] = df_players['name'].apply(lambda x: x.replace('â€', ''))
df_players['name'] = df_players['name'].apply(lambda x: x.replace('†', ''))
df_players['name'] = df_players['name'].apply(lambda x: x.replace('\xa0', ''))
df_players.head(10)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendis,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,"Blessed with a compact technique, an aggressiv..."


In [60]:
df_players.to_csv('player_info.csv', index = False)